# TTC Subway Delay Logs (2023) — Data-Cleaning Notebook

**Goal:** transform the raw 2023 Excel file into an analysis-ready CSV/parquet file.

Key cleaning tasks  
1. Standardize column names  
2. Parse `date` + `time` into a single `timestamp`  
3. Normalize text fields (`station`, `line`, `bound`)  
4. Ensure numeric columns are valid  
5. Remove duplicates  
6. Export cleaned dataset

In [4]:
# 1. Load raw data
import pandas as pd

# Use absolute path so the cell works no matter where this notebook sits
FILE_PATH = r"D:\Github\TahmidBuilds\ttc-subway-delay-cleaning\data\raw\ttc-subway-delay-data-2023.xlsx"

df = pd.read_excel(FILE_PATH)
df.head()


,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,2023-01-01,02:22,Sunday,MUSEUM STATION,MUPAA,3,9,S,YU,5931
1,2023-01-01,02:30,Sunday,KIPLING STATION,MUIS,0,0,E,BD,5341
2,2023-01-01,02:33,Sunday,WARDEN STATION,SUO,0,0,W,BD,0
3,2023-01-01,03:17,Sunday,KEELE STATION,MUIS,0,0,NaN,BD,0
4,2023-01-01,07:16,Sunday,BATHURST STATION,MUIS,0,0,NaN,BD,0


### 1️⃣  Standardize column names  
We’ll strip whitespace, make them lowercase, and replace spaces with underscores so they’re Python-friendly.


In [5]:
# 2. Clean column names
df.columns = (
    df.columns
      .str.strip()        # remove leading/trailing spaces
      .str.lower()        # lowercase
      .str.replace(" ", "_")
)

df.columns


Index(['date', 'time', 'day', 'station', 'code', 'min_delay', 'min_gap',
       'bound', 'line', 'vehicle'],
      dtype='object')

### 2️⃣  Parse `date` and `time`, create full `timestamp`

In [6]:
# Parse dates & times, then combine
df["date"] = pd.to_datetime(df["date"], errors="coerce")  # coerce invalid strings to NaT
df["time"] = pd.to_datetime(df["time"], format="%H:%M", errors="coerce").dt.time

# Combine into single column for easy filtering & de-duplication later
df["timestamp"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str))

df[["date", "time", "timestamp"]].head()

,date,time,timestamp
0,2023-01-01,02:22:00,2023-01-01 02:22:00
1,2023-01-01,02:30:00,2023-01-01 02:30:00
2,2023-01-01,02:33:00,2023-01-01 02:33:00
3,2023-01-01,03:17:00,2023-01-01 03:17:00
4,2023-01-01,07:16:00,2023-01-01 07:16:00


### 3️⃣  Normalize text fields (`station`, `line`, `bound`)

In [7]:
# 4. Tidy up text columns
df["station"] = df["station"].str.strip().str.title()   # "MUSEUM STATION" → "Museum Station"
df["line"]    = df["line"].str.strip().str.upper()      # ensure YU / BD etc.
df["bound"]   = df["bound"].str.strip().str.upper()     # N / S / E / W

df[["station", "line", "bound"]].head()


,station,line,bound
0,Museum Station,YU,S
1,Kipling Station,BD,E
2,Warden Station,BD,W
3,Keele Station,BD,NaN
4,Bathurst Station,BD,NaN


### 4️⃣  Ensure numeric columns are valid

In [8]:
numeric_cols = ["min_delay", "min_gap"]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")   # strings → NaN
    # Replace negative values (shouldn't exist) with NA
    df.loc[df[col] < 0, col] = pd.NA

df[numeric_cols].describe()


,min_delay,min_gap
count,22949.000000,22949.000000
mean,2.970892,4.370387
std,9.295641,10.353832
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,4.000000,8.000000
max,423.000000,428.000000


### 5️⃣  Drop duplicate delay records  
We consider a duplicate as having the **same timestamp, station, and code**.


In [9]:
before = len(df)
df = df.drop_duplicates(subset=["timestamp", "station", "code"])
after = len(df)

print(f"Rows before: {before:,}  |  after removing duplicates: {after:,}")


Rows before: 22,949  |  after removing duplicates: 22,926


## ✅ Cleaning Summary  

| Step | Action | Result |
|------|--------|--------|
| 1 | Column names standardized | Easy referencing in code |
| 2 | Combined `date` + `time` → `timestamp` | Simplified time-series ops |
| 3 | Text fields normalized | Consistent casing/spaces |
| 4 | Numeric validation | No negative or non-numeric delays |
| 5 | Duplicate removal | {{before-after counts above}} rows deleted |
| 6 | Export | CSV + Parquet saved in `data/clean/` |

In [11]:

# 1. Quick null summary
nulls = df.isna().sum().sort_values(ascending=False)
print(nulls)

# 2. Replace missing text fields with 'Unknown' for grouping visuals
text_cols = ["bound", "line", "vehicle"]
df[text_cols] = df[text_cols].fillna("Unknown")

# 3. Decide whether to keep or drop 0-minute rows
nonzero = df[df["min_delay"] > 0]      # for duration analyses

bound        8507
line           48
date            0
time            0
day             0
station         0
code            0
min_delay       0
min_gap         0
vehicle         0
timestamp       0
dtype: int64
